In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import patches

from klampt import *
from klampt.plan.cspace import CSpace,MotionPlan
from klampt.math import vectorops

class Circle:
    """A circle geometry.  Can collide with circles or rectangles."""
    def __init__(self,center,radius):
        self.center = center
        self.radius = radius
    def contains(self,p):
        return vectorops.distance(self.center,p) <= self.radius
    def collides(self,other):
        if isinstance(other,Circle):
            return vectorops.distance(self.center,other.center) <= self.radius+other.radius
        elif isinstance(other,Rectangle):
            return other.collides(self)
        else:
            raise ValueError("Circle can only collide with Circle or Rectangle")
    def draw_matplotlib(self,ax,**args):
        ax.add_patch(patches.Circle(self.center,self.radius,**args))
    
class Rectangle:
    def __init__(self,bmin,bmax):
        self.bmin = bmin
        self.bmax = bmax
    def contains(self,p):
        return (self.bmin[0] <= p[0] <= self.bmax[0]) and (self.bmin[1] <= p[1] <= self.bmax[1])
    def collides(self,other):
        if isinstance(other,Circle):
            closest = (max(self.bmin[0],min(other.center[0],self.bmax[0])),
                       max(self.bmin[1],min(other.center[1],self.bmax[1])))
            return other.contains(closest)
        elif isinstance(other,Rectangle):
            return (other.bmin[0] <= self.bmin[0] != other.bmax[0] <= self.bmax[0]) and (other.bmin[1] <= self.bmin[1] != other.bmax[1] <= self.bmax[1])
        else:
            raise ValueError("Rectangle can only collide with Circle or Rectangle")
    def draw_matplotlib(self,ax,**args):
        ax.add_patch(patches.Rectangle(self.bmin,self.bmax[0]-self.bmin[0],self.bmax[1]-self.bmin[1],**args))

class FourDiskCSpace(CSpace):
    def __init__(self,Rs,obstacles):
        CSpace.__init__(self)
        assert len(Rs) == 4,"Need to provide four radii"
        #configure settings in CSpace class
        self.setBounds([(0,1)]*8)
        self.eps = 0.01  #collision checking resolution
        
        #my class's settings
        self.radii = Rs
        self.obstacles = obstacles
        self.robot_circles = [Circle((0,0),r) for r in Rs]
    
    def sample(self):
        """TODO: return a randomly sampled configuration"""
        return [0.0]*8
    
    def feasible(self,q):
        """TODO: perform collision checks"""
        return True
    
    def visible(self,a,b):
        """TODO: perform collision checks on the straight line path"""
        return True
    

In [ ]:
radii = [0.05,0.075,0.03,0.08]
empty_obstacles = []
narrow_passage_obstacles = [Rectangle([0.3,0],[0.7,0.4]),Rectangle([0.3,0.6],[0.7,1.0])]
big_circle_obstacles = [Circle((0.5,0.5),0.32)]
#infeasible problems
too_big_circle_obstacles = [Circle((0.5,0.5),0.38)]
barrier_obstacles = [Rectangle([0.5,0.05],[0.51,0.95])]

#x-y swap problem
start_config = [0.1,0.1,  0.1,0.9,  0.9,0.1,  0.9,0.9]
goal_config = [0.9,0.15,  0.9,0.85,  0.1,0.15,  0.1,0.85]

#TODO: play around with the obstacle selection here
space = FourDiskCSpace(radii,empty_obstacles)

#TODO: play around with the planner parameters here
planner = MotionPlan(space,type="prm",connectionThreshold=0.5)
#planner = MotionPlan(space,type="prm",connectionThreshold=2.0)
#planner = MotionPlan(space,type="rrt",bidirectional=True,connectionThreshold=0.5)
#planner = MotionPlan(space,type="rrt",bidirectional=True,connectionThreshold=2.0)
optimizing = False

#the below planners are optimizing. Set optimizing=True to make full use of these
#optimizing = True
#planner = MotionPlan(space,type="prm",bidirectional=True,connectionThreshold=2,shortcut=True)
#planner = MotionPlan(space,type="rrt",bidirectional=True,connectionThreshold=2,shortcut=True)
#planner = MotionPlan(space,type="lazyrrg*")
#planner = MotionPlan(space,type="sbl",connectionThreshold=2,restart=True,shortcut=True)

#configure the start and goal of the planner
planner.setEndpoints(start_config,goal_config)

def test_plan(planner,tmax,optimizing=False):
    """This subroutine runs the planner for up to tmax seconds"""
    import time
    increment = 100                  #there is a little overhead for each planMore call, so it is best not to set the increment too low
    t0 = time.time()
    path = []
    print("Beginning planning...")
    pathFound = False
    while time.time() - t0 < tmax:   
        planner.planMore(increment)
        path = planner.getPath()
        if path:
            if optimizing and not pathFound:
                print("First solution found,",len(path),"milestones")
                print("Took time",time.time()-t0)
            elif not optimizing:
                print("Solved, path has",len(path),"milestones")
                print("Took time",time.time()-t0)
                break
            pathFound = True
    if not path:
        print("No path found")
    elif optimizing:
        print("Solved, path has",len(path),"milestones, and was optimized")
        print("Took time",time.time()-t0)
    return path

path = test_plan(planner,tmax=5.0,optimizing=optimizing)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
import numpy as np
import random

def plot_solution(space,qstart,qgoal,planner):
    plt.figure(figsize=(12,12))
    plt.axis('equal')
    plt.xlim(0,1)
    plt.ylim(0,1)
    for o in space.obstacles:
        o.draw_matplotlib(plt.gca(),color='k')
    nrobots = len(space.robot_circles)
    colors = [(random.uniform(0,1),random.uniform(0,1),random.uniform(0,1)) for i in range(nrobots)]
    for i,c in enumerate(space.robot_circles):
        c.center = qstart[i*2:i*2+2]
        c.draw_matplotlib(plt.gca(),color=colors[i])
        c.center = qgoal[i*2:i*2+2]
        c.draw_matplotlib(plt.gca(),color=colors[i],fill=False)
    path = planner.getPath()
    if path:
        for i in range(nrobots):
            x = [q[i*2] for q in path]
            y = [q[i*2+1] for q in path]
            plt.plot(x,y,color=colors[i])
    else:
        V,E = planner.getRoadmap()
        print("Roadmap has",len(V),"vertices and",len(E),"edges")
        for i in range(nrobots):
            data = np.zeros((len(E),2,2))
            for j,e in enumerate(E):
                q = V[e[0]]
                p = V[e[1]]
                data[j,:,0] = [q[i*2],p[i*2]]
                data[j,:,1] = [q[i*2+1],p[i*2+1]]
            plt.gca().add_collection(LineCollection(data,color=colors[i]))
plot_solution(space,start_config,goal_config,planner)
plt.show()